In [1]:
from fastai.text import * 
import torch
import pandas as pd

In [6]:
df = pd.read_csv('../data/data_sample-500k-preprocessed.csv')

In [7]:
train=df[['label','text']]

In [8]:
df = pd.read_csv('../data/data_sample-100k-preprocessed.csv')

In [9]:
valid=df[['label','text']]

In [18]:
data_lm = TextLMDataBunch.from_df('./', train, valid)

In [19]:
data_clas = TextClasDataBunch.from_df('./', train, valid, vocab=data_lm.train_ds.vocab, bs=32)

In [12]:
data_lm.save('data_lm_preprocessed_export.pkl')
data_clas.save('data_clas_preprocessed_export.pkl')

In [16]:
path = './'
data_lm = load_data(path, fname='data_lm_preprocessed_export.pkl')
data_clas = load_data(path, fname='data_lm_preprocessed_export.pkl', bs=16)

In [23]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.232440,2.067320,0.648287,17:45


In [24]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.859634,1.805465,0.681374,19:55


In [25]:
learn.save_encoder('ft_enc_preprocessed')

In [26]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc_preprocessed')

In [27]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.581616,0.520620,0.743640,07:20


In [28]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.475569,0.428442,0.800450,08:32


In [29]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.452938,0.404077,0.812720,18:00


In [30]:
learn.save("trained_model_preprocessed")

In [22]:
learn = learn.load("trained_model_preprocessed")

RuntimeError: Error(s) in loading state_dict for SequentialRNN:
	size mismatch for 0.module.encoder.weight: copying a param with shape torch.Size([60003, 400]) from checkpoint, the shape in current model is torch.Size([60004, 400]).
	size mismatch for 0.module.encoder_dp.emb.weight: copying a param with shape torch.Size([60003, 400]) from checkpoint, the shape in current model is torch.Size([60004, 400]).

In [31]:
preds = []
for i in range (0,100000):
  preds.append(learn.predict(learn.data.valid_ds.x[i])[1].item())
  if(i%1000==0):
    print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000


In [36]:
preds = np.array(preds)

In [44]:
labels = learn.data.valid_ds.y.items

In [47]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score

In [48]:
labels_data = labels
results_data = preds
print(classification_report(labels_data,results_data))
print(f1_score(labels_data,results_data))
print(precision_score(labels_data,results_data))
print(recall_score(labels_data,results_data))
print(accuracy_score(labels_data,results_data))

             precision    recall  f1-score   support

          0       0.81      0.86      0.84     55900
          1       0.81      0.75      0.78     44100

avg / total       0.81      0.81      0.81    100000

0.7792430855381575
0.8096961103097572
0.7509977324263039
0.81235
